In [1]:
import pandas as pd

# Load all 3 main CSV files
sales = pd.read_csv("data/sales_train_validation.csv")
calendar = pd.read_csv("data/calendar.csv")
prices = pd.read_csv("data/sell_prices.csv")

# Check their shapes
print("Sales:", sales.shape)
print("Calendar:", calendar.shape)
print("Prices:", prices.shape)



Sales: (30490, 1919)
Calendar: (1969, 14)
Prices: (6841121, 4)


In [2]:
print(sales.shape)
sales.head()


(30490, 1919)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [3]:
import pandas as pd

# List of ID columns to keep
id_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

# Melt the dataframe to turn d_1...d_1913 into rows
sales_long = pd.melt(
    sales,
    id_vars=id_columns,
    var_name='d',           # this will be the day (d_1, d_2...)
    value_name='sales'      # this will be the value sold
)

# Check the reshaped data
sales_long.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [4]:
# Merge the reshaped sales data with calendar info using 'd'
sales_with_dates = pd.merge(sales_long, calendar, on='d', how='left')

# Preview important columns
sales_with_dates[['item_id', 'store_id', 'd', 'date', 'weekday', 'event_name_1', 'sales']].head()



,item_id,store_id,d,date,weekday,event_name_1,sales
0,HOBBIES_1_001,CA_1,d_1,2011-01-29,Saturday,NaN,0
1,HOBBIES_1_002,CA_1,d_1,2011-01-29,Saturday,NaN,0
2,HOBBIES_1_003,CA_1,d_1,2011-01-29,Saturday,NaN,0
3,HOBBIES_1_004,CA_1,d_1,2011-01-29,Saturday,NaN,0
4,HOBBIES_1_005,CA_1,d_1,2011-01-29,Saturday,NaN,0


In [8]:
# Choose a small subset: 3 items from CA_1 store
sample_sales = sales[sales['store_id'] == 'CA_1'].head(3)


In [9]:
# Reshape small sample
sample_long = pd.melt(
    sample_sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)


In [10]:
sample_with_dates = pd.merge(sample_long, calendar, on='d', how='left')


In [11]:
sample_full = pd.merge(
    sample_with_dates,
    prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

# View sample
sample_full[['item_id', 'store_id', 'date', 'sell_price', 'sales']].head()


,item_id,store_id,date,sell_price,sales
0,HOBBIES_1_001,CA_1,2011-01-29,NaN,0
1,HOBBIES_1_002,CA_1,2011-01-29,NaN,0
2,HOBBIES_1_003,CA_1,2011-01-29,NaN,0
3,HOBBIES_1_001,CA_1,2011-01-30,NaN,0
4,HOBBIES_1_002,CA_1,2011-01-30,NaN,0


In [21]:
# Required Libraries
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# -------------------------------------------
# STEP 1: Select a smaller sample to avoid memory issues
sample_sales = sales[sales['store_id'] == 'CA_1'].head(100)

# Reshape from wide to long
sample_long = pd.melt(
    sample_sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Merge with calendar
sample_with_dates = pd.merge(sample_long, calendar, on='d', how='left')

# Merge with prices
sample_full = pd.merge(
    sample_with_dates,
    prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

# -------------------------------------------
# STEP 2: Prepare Data for Training
df = sample_full.copy()

# Fill missing values safely (no warning)
df['event_name_1'] = df['event_name_1'].fillna('None')
df['sell_price'] = df['sell_price'].fillna(0)

# Convert categorical values to numeric codes
df['weekday'] = df['weekday'].astype('category').cat.codes
df['event_name_1'] = df['event_name_1'].astype('category').cat.codes

# Features and target
features = ['sell_price', 'weekday', 'event_name_1']
X = df[features]
y = df['sales']

# -------------------------------------------
# STEP 3: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Regressor
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Test RMSE: {rmse:.2f}")

print(f"Test RMSE: {rmse:.2f}")


Test RMSE: 2.74
Test RMSE: 2.74


In [7]:
# Required Libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

sales = pd.read_csv("data/sales_train_validation.csv")
calendar = pd.read_csv("data/calendar.csv")
prices = pd.read_csv("data/sell_prices.csv")

# STEP 1: Select a smaller sample to avoid memory issues
sample_sales = sales[sales['store_id'] == 'CA_1'].head(5000)

# Reshape from wide to long
sample_long = pd.melt(
    sample_sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Merge with calendar
sample_with_dates = pd.merge(sample_long, calendar, on='d', how='left')

# Merge with prices
sample_full = pd.merge(
    sample_with_dates,
    prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

# -------------------------------------------
# STEP 2: Prepare Data for Training
df = sample_full.copy()

# Fill missing values safely (no warning)
df['event_name_1'] = df['event_name_1'].fillna('None')
df['sell_price'] = df['sell_price'].fillna(0)

# Convert categorical values to numeric codes
df['weekday'] = df['weekday'].astype('category').cat.codes
df['event_name_1'] = df['event_name_1'].astype('category').cat.codes

# Features and target
features = ['sell_price', 'weekday', 'event_name_1']
X = df[features]
y = df['sales']

# -------------------------------------------
# STEP 3: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mse_rf ** 0.5
print(f"Random Forest RMSE: {rmse_rf:.2f}")


Random Forest RMSE: 3.70


In [6]:
# Required Libraries
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# -------------------------------------------
# STEP 1: Select a smaller sample to avoid memory issues
sample_sales = sales[sales['store_id'] == 'CA_1'].head(5000)

# Reshape from wide to long
sample_long = pd.melt(
    sample_sales,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='sales'
)

# Merge with calendar
sample_with_dates = pd.merge(sample_long, calendar, on='d', how='left')

# Merge with prices
sample_full = pd.merge(
    sample_with_dates,
    prices,
    on=['store_id', 'item_id', 'wm_yr_wk'],
    how='left'
)

# -------------------------------------------
# STEP 2: Prepare Data for Training
df = sample_full.copy()

# Fill missing values safely (no warning)
df['event_name_1'] = df['event_name_1'].fillna('None')
df['sell_price'] = df['sell_price'].fillna(0)

# Convert categorical values to numeric codes
df['weekday'] = df['weekday'].astype('category').cat.codes
df['event_name_1'] = df['event_name_1'].astype('category').cat.codes

# Features and target
features = ['sell_price', 'weekday', 'event_name_1']
X = df[features]
y = df['sales']

# -------------------------------------------
# STEP 3: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Clean GPU-accelerated XGBoost
model = xgb.XGBRegressor(
    tree_method='hist',
    device='cuda',
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Test RMSE (XGBoost GPU): {rmse:.2f}")




Test RMSE (XGBoost GPU): 3.75
